<a href="https://colab.research.google.com/github/Alf-caput/P4_AlgoritmoGenetico/blob/dev3/P4_AlgGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PRÁCTICA 4: Algoritmo genético

### Ana Robledano, Miguel Egido, Alfredo Robledano.

1.- Encontrar los máximos de la función "picos"

$$z = f(x,y) = 3(1-x)^2e^{-(x^2)-(y+1)^2}-10(x/5-x^3-y^5)e^{-x^2-y^2}-1/3e^{-(x+1)^2-y^2}$$

En el dominio $$\{x,y\} \in [-100,100]$$


In [243]:
import random
from deap import base, creator, tools

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

random.seed(42)

VALID_RANGE = (-100, 100)
INDIVIDUAL_SIZE = 2
POPULATION_SIZE = 32

toolbox = base.Toolbox()

toolbox.register("attr_uniform", random.uniform, *VALID_RANGE)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_uniform, INDIVIDUAL_SIZE)
toolbox.register("population", tools.initRepeat, list, toolbox.individual, POPULATION_SIZE)

population = toolbox.population()
print(population)

[[27.885359691576753, -94.99784895546661], [-44.99413632617615, -55.35785237023545], [47.29424283280247, 35.33989748458225], [78.43591354096907, -82.61223347411678], [-15.61563606294591, -94.04055611238593], [-56.27240503927933, 1.0710576206724767], [-94.69280606322728, -60.2324698626703], [29.976887555904653, 8.98829612064334], [-55.911875591860664, 17.85313677518174], [61.88609133556534, -98.7002480643878], [61.163850366561576, 39.62787899764538], [-31.949896696401623, -68.90410003764369], [91.44261444135623, -32.68109097747465], [-81.45083132397042, -80.65672463330719], [69.49887326949195, 20.745206273378216], [61.42565465487604, 45.94635733876359], [7.245618290940143, 94.62315279587412], [-24.29312455832931, 10.408126254645396], [65.88093285059898, 23.703950472849215], [72.34138006215545, 15.470429051352411], [40.914367242984696, -90.83512326886756], [-54.42034486969063, -42.12240727957857], [-84.04160461527451, -53.44182272779396], [-79.79971411805417, -44.40527937798158], [27.136

In [244]:
import math
from functools import partial

F = lambda x, y: 3*(1-x)**2*math.exp(-(x**2)-(y+1)**2)-10*(x/5-x**3-y**5)*math.exp(-x**2-y**2)-1/3*math.exp(-(x+1)**2-y**2)

def evaluate(individual, target_function, gene_range):
    for gene in individual:
        if not (gene_range[0] <= gene <= gene_range[1]):
            return -1,
    return target_function(*individual),

toolbox.register("evaluate", partial(evaluate, target_function=F, gene_range=VALID_RANGE))

In [245]:
MU = 0 # mean
SIGMA = 5 # standard deviation
INDPB = 0.1 # independent probability for each atribute to be mutated
TOURNAMENT_SIZE = 3 # tournament size

toolbox.register("mate", tools.cxOnePoint)
toolbox.register("mutate", tools.mutGaussian, MU, SIGMA, INDPB)
toolbox.register("select", tools.selTournament, tournsize=TOURNAMENT_SIZE)

In [246]:
from deap import algorithms
import numpy as np

CXPB = 0.5 # probability of mating 2 individuals
MUTPB = 0.2 # probabilty of mutating an individual
NGEN = 20 # number of generations

pop = toolbox.population()

hof = tools.HallOfFame(1)
stats = tools.Statistics(lambda ind: ind.fitness.values)

stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

logbook = tools.Logbook()
pop, logbook = algorithms.eaSimple(pop, toolbox, cxpb=CXPB, mutpb=MUTPB, ngen=NGEN, stats=stats, halloffame=hof, verbose=True)

gen	nevals	avg	std	min	max
0  	32    	0  	0  	0  	0  


TypeError: object of type 'int' has no len()

2

2.- Dada la [Función de Rastringin](https://en.wikipedia.org/wiki/Rastrigin_function#:~:text=In%20mathematical%20optimization%2C%20the%20Rastrigin,has%20been%20generalized%20by%20Rudolph.), con A = 10 y  $$\{xi\} \in [-5.12,5.12]$$

Implementa dicha función en Python y minimízala para n=10

Compara el funcionamiento del Algoritmo Genético para cruce con un punto con cruce con dos puntos

### Instalación de librerías para resolver Algoritmos Genéticos con Python

In [ ]:
#!pip install deap
#!pip install matplotlib
#!pip install numpy
#!pip install scipy
#!pip install scoop